In [ ]:
"""
=================================================================
This is a template video for demonstrating Analog to Digital
signal conversion graphically using the Manim library.

Title: Analog to Digital Conversion Visualization
Author: P Michael Crocco
Affiliation: Office of the Dean of Engineering, Monash University
Date: 10 May 2024
Version: 1.0
=================================================================
"""

from manim import *
import numpy as np
from scipy.optimize import fsolve

# set background color (delete to default to BLACK)
config.background_color = WHITE

class AnalogToDigital(Scene):

    def construct(self):

        # Axes and harmonic
        vert_shift = 0.5
        amplitude = 0.5

        # Define the harmonic function
        def harmonic(x):
            return amplitude * np.sin(x) + vert_shift
        
        # Define the axes, supress tickmarks
        ax = Axes(x_range=[-0.5, 4 * PI + 1],
                  y_range=[-0.1, 1.1],
                  x_axis_config={"include_ticks": False},
                  y_axis_config={"include_ticks": False},
                  axis_config={"color": BLACK}
        )
        graph = ax.plot(lambda x: harmonic(x), color=BLACK, x_range=[0, 4 * PI])

        # Draw in Axes
        self.play(
            FadeIn(ax)
        )
        self.wait

        # Draw Sine wave
        self.play(
            FadeIn(graph, run_time = 1.0),
            rate_func=linear
        )

        # Insert a label (note, default color is WHITE)
        label1 = Tex("Signal", color=BLACK).next_to(graph, RIGHT)
        self.play(FadeIn(label1))

        self.wait

        # Set up digital thresholds
        threshold = 0.7
        upper_threshold = vert_shift + threshold * amplitude
        lower_threshold = vert_shift - threshold * amplitude

        # Draw lines at thresholds
        upper_bar = DashedLine(start = ax.coords_to_point(0, upper_threshold), end = ax.coords_to_point(4 * PI + 1, upper_threshold,0), color=PINK)
        lower_bar = DashedLine(start = ax.coords_to_point(0, lower_threshold), end = ax.coords_to_point(4 * PI + 1, lower_threshold,0), color=PINK)
        self.play(Create(upper_bar))
        self.play(Create(lower_bar))

        self.wait

        # Find first two intersections
        def find_intersections(level):
            first_guess = fsolve(lambda x: harmonic(x) - level, np.pi/4)
            second_guess = fsolve(lambda x: harmonic(x) - level, 5*PI/4)
            return first_guess, second_guess
        
        first_intersection, second_intersection = find_intersections(upper_threshold)

        # Draw points on intersections
        first_point = Dot(ax.coords_to_point(first_intersection[0], harmonic(first_intersection[0])), color=RED)
        second_point = Dot(ax.coords_to_point(second_intersection[0], harmonic(second_intersection[0])), color=RED)
        self.play(Create(first_point))
        self.play(Create(second_point))

        
        # Draw the line between the intersection points
        bar = Line(start=ax.coords_to_point(first_intersection[0], harmonic(first_intersection[0])),
                   end=ax.coords_to_point(second_intersection[0], harmonic(second_intersection[0])),
                   color=YELLOW, stroke_width = 10)
        self.play(Create(bar), run_time=1.0)

        self.wait(1)

        # Move the line up to the value of y=1.0
        midpoint = (bar.get_start() + bar.get_end()) /2

        target_y = ax.coords_to_point(0,1)[1]
        current_y = midpoint[1]
        shift_vector = np.array([0, target_y - current_y, 0])

        self.play(bar.animate.shift(shift_vector), run_time=1.0)

        #Copy these lines in the x-direction and place accordingly
        self.wait(1)

        bar_length = second_intersection[0] - first_intersection[0]

        bars = VGroup()
       
        initial_x = -PI/2
     
        for i in range(1,5):
            x_center =  initial_x + i * PI
            
            if i % 2 == 1:
                y_position = 1
            else:
                y_position = 0
                         
            start_point = ax.coords_to_point(x_center - bar_length / 2, y_position)
            end_point = ax.coords_to_point(x_center + bar_length / 2, y_position)
            
            bar = Line(
                start=start_point,
                end=end_point,
                color=YELLOW, stroke_width=10

            )
            bars.add(bar) 

        self.play(Create(bars), run_time = 0.5)
        self.wait()

        #draw rectangle
        first_bar_end = bars[0].get_end()
        second_bar_start = bars[1].get_start()

        rectangle = Rectangle(
            width=abs(second_bar_start[0] - first_bar_end[0]),
            height=abs(second_bar_start[1] - first_bar_end[1]),
            color=BLUE, fill_opacity=0.5
        )

        rectangle.move_to((first_bar_end + second_bar_start) /2)

        self.play(FadeIn(rectangle))

        self.wait(3)


# Preview the video set -qX to get appropriate quality
%manim -qh -v WARNING AnalogToDigital
# uncomment the following will open the manim folder
# %manim -qm -f AnalogToDigital